<a href="https://colab.research.google.com/github/Shreya0302-source/SciML-Implementations/blob/main/(S)B_approach_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ziq_sciml_challenge_path = kagglehub.competition_download('ziq-sciml-challenge')
shreyajoshi03_thermal_management_test_dataset_path = kagglehub.dataset_download('shreyajoshi03/thermal-management-test-dataset')

print('Data source import complete.')


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [ ]:
torch.manual_seed(42)

In [ ]:
# Neural Network
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 256), nn.Tanh(),
            *[nn.Linear(256, 256), nn.Tanh()] * 7,
            nn.Linear(256, 1)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
# Heat generation function f(x, y)
def f_xy(x, y, epsilon=1e-4):
    term1 = 2 * epsilon * (-x + torch.exp(2 * (x - 1) / epsilon))
    term2 = x * y**2 + 6 * x * y
    term3 = -x * torch.exp(3 * (y - 1) / epsilon)
    term4 = -y**2 * torch.exp(2 * (x - 1) / epsilon)
    term5 = 2 * y**2 - 6 * y * torch.exp(2 * (x - 1) / epsilon)
    term6 = -2 * torch.exp(3 * (y - 1) / epsilon)
    term7 = torch.exp(2 * x + 3 * y - 5 * epsilon)
    return term1 + term2 + term3 + term4 + term5 + term6 + term7

In [ ]:
# PDE residual (Corrected version)
def pde_loss(model, x, epsilon=1e-4, bx=2.0, by=3.0):
    x.requires_grad_(True)
    u = model(x)
    u_grad = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x, u_y = u_grad[:, 0], u_grad[:, 1]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0]
    u_yy = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1]
    f = f_xy(x[:, 0], x[:, 1], epsilon)  # Properly call f_xy
    residual = -epsilon * (u_xx + u_yy) + bx * u_x + by * u_y - f
    return torch.mean(residual**2), residual

In [ ]:
# Boundary loss
def bc_loss(model, x_bc):
    u_bc = model(x_bc)
    return torch.mean(u_bc**2)

In [ ]:
# Generate training points
def generate_points(n_pde=20000, n_bc=800):
    # Collocation points inside domain
    x_pde = torch.rand(n_pde, 2)

    # Boundary points (200 per side)
    x_bc = torch.cat([
        torch.stack([torch.linspace(0, 1, 200), torch.zeros(200)], dim=1),  # y = 0
        torch.stack([torch.linspace(0, 1, 200), torch.ones(200)], dim=1),   # y = 1
        torch.stack([torch.zeros(200), torch.linspace(0, 1, 200)], dim=1),  # x = 0
        torch.stack([torch.ones(200), torch.linspace(0, 1, 200)], dim=1)    # x = 1
    ])
    return x_pde, x_bc

In [ ]:
# Adaptive sampling
def adaptive_sample(model, x_pde, n_keep=5000, n_add=5000):
    _, residuals = pde_loss(model, x_pde)
    residuals = residuals.abs().detach()
    _, indices = torch.topk(residuals, n_keep)
    x_keep = x_pde[indices]
    x_new = torch.rand(n_add, 2)
    return torch.cat([x_keep, x_new])

In [ ]:
# Training function
def train_pinn(model, epochs_adam=20000, epochs_lbfgs=2000):
    x_pde, x_bc = generate_points()
    optimizer_adam = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer_adam, step_size=5000, gamma=0.1)

    print("Training with Adam...")
    for epoch in range(epochs_adam):
        optimizer_adam.zero_grad()
        loss_pde, _ = pde_loss(model, x_pde)
        loss_bc = bc_loss(model, x_bc)
        loss = loss_pde + 10.0 * loss_bc
        loss.backward()
        optimizer_adam.step()
        scheduler.step()

        if (epoch + 1) % 1000 == 0:
            print(f"Adam Epoch {epoch + 1}/{epochs_adam}, Loss: {loss.item():.6f}, "
                  f"PDE: {loss_pde.item():.6f}, BC: {loss_bc.item():.6f}")
            x_pde = adaptive_sample(model, x_pde)

        if epoch > 10000 and loss.item() < 0.05:
            break

    optimizer_lbfgs = torch.optim.LBFGS(model.parameters(), lr=0.8, max_iter=50, history_size=100)

    def closure():
        optimizer_lbfgs.zero_grad()
        loss_pde, _ = pde_loss(model, x_pde)
        loss_bc = bc_loss(model, x_bc)
        loss = loss_pde + 10.0 * loss_bc
        loss.backward()
        return loss

    print("\nSwitching to L-BFGS...")
    for epoch in range(epochs_lbfgs // 50):
        loss = optimizer_lbfgs.step(closure)
        if (epoch + 1) % 25 == 0:
            print(f"L-BFGS Epoch {(epoch + 1) * 50}/{epochs_lbfgs}, Loss: {loss.item():.6f}")

In [ ]:
# Plotting
def plot_solution(model):
    x = torch.linspace(0, 1, 100)
    y = torch.linspace(0, 1, 100)
    X, Y = torch.meshgrid(x, y, indexing='ij')
    XY = torch.stack([X.flatten(), Y.flatten()], dim=1)

    with torch.no_grad():
        U = model(XY).reshape(100, 100).numpy()

    plt.figure(figsize=(8, 6))
    plt.contourf(X.numpy(), Y.numpy(), U, levels=50, cmap='jet')
    plt.colorbar(label='Temperature (u)')
    plt.title('PINN Solution: Temperature Distribution')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [ ]:
model = PINN()

In [ ]:
train_pinn(model)

In [ ]:
plot_solution(model)

In [ ]:
test_data = pd.read_csv("/kaggle/input/ziq-sciml-challenge/test.csv")
x_test = torch.tensor(test_data["x"].values.reshape(-1, 1), dtype=torch.float32)
y_test = torch.tensor(test_data["y"].values.reshape(-1, 1), dtype=torch.float32)

In [ ]:
# Make predictions
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    u_pred = model(x_test, y_test)

# Convert predictions to numpy array
u_pred_np = u_pred.numpy()

In [ ]:
# Create a DataFrame with the predictions
submission_df = pd.DataFrame({
    "ID": test_data["ID"],
    "x": test_data["x"],
    "y": test_data["y"],
    "u_pred": u_pred_np.flatten()
})

# Save the DataFrame to a CSV file
submission_df.to_csv("/kaggle/working/submission.csv", index=True)

# Print the predictions
print(u_pred_np)